# Day3 Pre-training Pipeline 演習
- 学習データの準備
- 言語モデルとは
- ニューラルネットワークを使用しない手法(uni-gram, bi-gram)
- Transformer以前のニューラルネットワークによる言語モデルのデザイン
    - MLP
    - RNN
- Transformerによる言語モデルのデザイン
    - Self-AttentionとFeedforward Networkの実装、並列化
    - GPT-2の実装

## 学習データの準備
ChatGPTのような大規模言語モデルの学習には文書データを大量に集める必要があります。
今回は夏目漱石著「吾輩は猫である」の文字データをインターネット上から取得して学習用に加工します。

青空文庫というサイトに「吾輩は猫である」の全文が公開されています

https://www.aozora.gr.jp/cards/000148/files/789_14547.html

In [ ]:
# Webページのダウンロード, 受講者が一斉に実行すると攻撃になるので注意, こちらで実行したダウンロード結果をneco.htmlをすでに用意しました
# !wget "https://www.aozora.gr.jp/cards/000148/files/789_14547.html" -O neco.html

In [ ]:
# ダウンロードしたデータの確認, 最初の2000文字を表示
open('neco.html', encoding="shift-jis").read()[:2000]

In [ ]:
import re
def clean_text(html_file_name):
    with open(html_file_name, encoding="shift-jis") as f:
        lines = f.readlines()
    texts = []
    for i, line in enumerate(lines):
        if i >= 25 and i <= 2359:
            #print(line)
            line = line.replace("<br />", "")
            line = line.replace("<ruby>", "")
            line = line.replace("</ruby>", "")
            line = line.replace("<rp>", "")
            line = line.replace("</rp>", "")
            line = line.replace("<rt>", "")
            line = line.replace("</rt>", "")
            line = line.replace("<rb>", "")
            line = line.replace("</rb>", "")

            line = re.sub(r'</em>', '', line)
            line = re.sub(r'<em.*">', '', line)
            line = re.sub(r'</div>', '', line)
            line = re.sub(r'<div.*">', '', line)

            line = line.strip("\n").strip(" ").strip("　")

            if i < 100:
                # print(i)
                # print(line)
                pass

            if line != "":
                texts.append(line)
    return texts

In [ ]:
# 最初の1データを表示
print(clean_text('neco.html')[0])

In [ ]:
with open('neco.txt', 'w') as f:
    for sentence in clean_text('neco.html'):
        f.write(sentence)
        f.write('\n')

## 言語モデルとは　　
大規模言語モデルの"言語モデル"とは、単語列の出現確率をモデル化したものです。  
確率を計算できるので、与えられた文章がよく見る文章(確率が高い)なのか、変な文章なのか(確率が低い)を判断することができたり、新たに文章を生成(確率に従ってくじ引きをする)することができます。
より良い言語モデルの開発のためには、データとどのように確率をモデル化するかのデザインが重要です。
### データについて
言語モデルはデータを元に学習するため、そのデータに有益な情報(例えば日本の歴史や法律に関する文章)が含まれていないと、単語自身の理解や単語同士の関係性を学習することができません。  
人間が本を読んだり、人との会話を通じて新しい知識を得たり、良い文章の書き方を学んだりするように、言語モデルもデータを通じて学習します。
### モデル化について
データを得たとしても、どのように確率をモデル化するかのデザインがうまくいかないと、良い言語モデルは作れません。
- データ中の文字を数え上げて前の1単語から次の1単語を予測するモデル
- ニューラルネットワークの一種であるMLPを用いて、前の数単語から次の1単語を予測するモデル
- Transformerを用いて、より長い文脈を考慮して次の単語を予測するモデル  

を開発します。

## ニューラルネットワークを使用しない、数え上げによる手法(uni-gram, bi-gram)の言語モデル

![N-gram](images/ngram.png)

### n-gram 言語モデル
$P\left(w_1, \ldots, w_m\right)=\prod_{i=1}^{i=m} P\left(w_i \mid w_1, \ldots, w_{i-1}\right) \approx \prod_{i=1}^{i=m} P\left(w_i \mid w_{i-n}, \ldots, w_{i-1}\right)$

#### uni-gramモデル
$P\left(w_1, \ldots, w_m\right) \approx \prod_{i=1}^{i=m} P(w_i)$

#### bi-gramモデル  
$P\left(w_1, \ldots, w_m\right)\approx \prod_{i=1}^{i=m} P\left(w_i \mid w_{i-1}\right)$

$P\left(w_{i} \mid w_{i-1}\right)=\frac{\operatorname{count}\left(w_{i}, w_{i-1}\right)}{\operatorname{count}\left(w_{i}\right)}$

In [ ]:
# https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/makemore/makemore_part1_bigrams.ipynb

In [ ]:
sentences = clean_text('neco.html')
all_text_data = ''.join(sentences)
# 最初の500文字を表示
print(all_text_data[:500])
print('------------------------')
print('全体の文字数: ', len(all_text_data))
print('文字の種類: ', len(set(all_text_data)))

In [ ]:
word_count = {}
for ch in all_text_data:
    word_count[ch] = word_count.get(ch, 0) + 1
# 出現頻度の高い文字を上位5個表示
print('出現頻度の高い文字')
print(sorted(word_count.items(), key = lambda kv: -kv[1])[:5])

# 過去の情報を考慮しないuni-gramモデル
print()
total_count = sum(word_count.values())
print(f"'猫'の確率 = '猫'の出現回数 ÷ 全体の文字数 =  {word_count['猫']} ÷ {total_count} = {word_count['猫'] / total_count}")
print(f"'吾輩は猫'の確率 = '吾'の確率 x '輩'の確率 x 'は'の確率 x '猫'の確率 = {word_count['吾'] / total_count} x {word_count['輩'] / total_count} x {word_count['は'] / total_count} x {word_count['猫'] / total_count} = {word_count['吾'] * word_count['輩'] * word_count['は'] * word_count['猫'] / total_count ** 4}")
print(f"'吾輩は犬'の確率 = '吾'の確率 x '輩'の確率 x 'は'の確率 x '犬'の確率 = {word_count['吾'] / total_count} x {word_count['輩'] / total_count} x {word_count['は'] / total_count} x {word_count['犬'] / total_count} = {word_count['吾'] * word_count['輩'] * word_count['は'] * word_count['犬'] / total_count ** 4}")

In [ ]:
# 過去の文脈を考慮しないuni-gramモデルによる生成
import torch
N = torch.zeros((len(set(all_text_data))+2), dtype=torch.int32)
chars = sorted(list(set(all_text_data)))
str_to_idx = {s:i+2 for i,s in enumerate(chars)}
str_to_idx['<S>'] = 0
str_to_idx['<E>'] = 1
idx_to_str = {i:s for s,i in str_to_idx.items()}

for sentence in sentences:
    chs = ['<S>'] + list(sentence) + ['<E>']
    for ch in chs:
        idx = str_to_idx[ch]
        N[idx] += 1

In [ ]:
str_to_idx['猫']

In [ ]:
idx_to_str[1759]

In [ ]:
P = (N[2:]).float()
P /= P.sum(0, keepdims=True)
g = torch.Generator().manual_seed(1)
context = '吾輩'
out = list(context)
print('入力: ', ''.join(out))
while True:
    idx = torch.multinomial(P, num_samples=1, replacement=True, generator=g).item() + 2
    print(idx, idx_to_str[idx])
    out.append(idx_to_str[idx])
    if len(out) >= 10:
        break
print('生成結果: ', ''.join(out[:-1]))

In [ ]:
bigram_count = {}
for sentence in sentences:
    chs = ['<S>'] + list(sentence) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        bigram_count[bigram] = bigram_count.get(bigram, 0) + 1
print('出現頻度の高い、2文字のペア')
print(sorted(bigram_count.items(), key = lambda kv: -kv[1])[:5])

import torch
N = torch.zeros((len(set(all_text_data))+2, len(set(all_text_data))+2), dtype=torch.int32)
chars = sorted(list(set(all_text_data)))
str_to_idx = {s:i+2 for i,s in enumerate(chars)}
str_to_idx['<S>'] = 0
str_to_idx['<E>'] = 1
idx_to_str = {i:s for s,i in str_to_idx.items()}

for sentence in sentences:
    chs = ['<S>'] + list(sentence) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        idx1 = str_to_idx[ch1]
        idx2 = str_to_idx[ch2]
        N[idx1, idx2] += 1

In [ ]:
bigram_count[('は', '猫')]

In [ ]:
# 1文字前を考慮するbi-gramモデル
print(f"'は→猫'('は'の後に'猫'が続く)の確率 = 'は猫'の出現回数 ÷ 'は'の出現回数 =  {N[str_to_idx['は'], str_to_idx['猫']]} ÷ {N.sum(1)[str_to_idx['は']]} = {N[str_to_idx['は'], str_to_idx['猫']] / N.sum(1)[str_to_idx['は']]}")
print(f"'吾輩は猫'の確率 = '吾'の確率 x '吾→輩'の確率 x '輩→は'の確率 x 'は→猫'の確率 = {word_count['吾'] / total_count} x {N[str_to_idx['吾'], str_to_idx['輩']] / N.sum(1)[str_to_idx['吾']]} x {N[str_to_idx['輩'], str_to_idx['は']] / N.sum(1)[str_to_idx['輩']]} x {N[str_to_idx['は'], str_to_idx['猫']] / N.sum(1)[str_to_idx['は']]} = {(word_count['吾'] / total_count) * (N[str_to_idx['吾'], str_to_idx['輩']] / N.sum(1)[str_to_idx['吾']]) * (N[str_to_idx['輩'], str_to_idx['は']] / N.sum(1)[str_to_idx['輩']]) * (N[str_to_idx['は'], str_to_idx['猫']] / N.sum(1)[str_to_idx['は']])}")
print(f"'吾輩は犬'の確率 = '吾'の確率 x '吾→輩'の確率 x '輩→は'の確率 x 'は→犬'の確率 = {word_count['吾'] / total_count} x {N[str_to_idx['吾'], str_to_idx['輩']] / N.sum(1)[str_to_idx['吾']]} x {N[str_to_idx['輩'], str_to_idx['は']] / N.sum(1)[str_to_idx['輩']]} x {N[str_to_idx['は'], str_to_idx['犬']] / N.sum(1)[str_to_idx['は']]} = {(word_count['吾'] / total_count) * (N[str_to_idx['吾'], str_to_idx['輩']] / N.sum(1)[str_to_idx['吾']]) * (N[str_to_idx['輩'], str_to_idx['は']] / N.sum(1)[str_to_idx['輩']]) * (N[str_to_idx['は'], str_to_idx['犬']] / N.sum(1)[str_to_idx['は']])}")

In [ ]:
# 1文字前を考慮するbi-gramモデルによる生成
P = (N).float()
P /= P.sum(1, keepdims=True)

g = torch.Generator().manual_seed(1)

context = '吾輩'
out = list(context)
print('入力: ', ''.join(out))
idx = str_to_idx[out[-1]]
while True:
    p = P[idx]
    idx = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(idx_to_str[idx])
    if idx == 1 or len(out) >= 100:
        break
print('生成結果: ', ''.join(out[:-1]))

## ニューラルネットワークを使用した言語モデル　　
言語モデルをデザインする際に、過去の文脈を考慮するとより自然な文章のモデリングができると考えられます。  
ただ、N-gramモデルのパラメータ数のオーダーは、$O\left(\left|V\right|^n\right)$　となり、過去の文脈が増えれば増えるほど組み合わせが膨大になります。  
過去の3文字を考慮するだけでも、今回の文字種類数だとパラメータ数(表のマス目の数) 3016 ** 4  = 82,741,873,217,536(約80兆)となり、今回の小説データの文字数(337,202)より多くなり、GPT-3のパラメータ数1750億を超えます。  
組み合わせ単位で数え上げているため、データ中に出現しない組み合わせがあると、その組み合わせの確率は0となり、その後の予測ができなくなってしまいます。  

また、組み合わせ別の数え上げでは、単語・文脈をそれぞれ独立したものとして扱っており、単語の意味や文脈を共有した表現が得られません。

解決策として組み合わせの表によるモデル化ではなく、ニューラルネットワークと単語ベクトルの表現を用いることでモデル化を行います。

ニューラルネットワークの学習の際に行う、次単語予測によって単語ベクトルには単語の意味や概念が付与され、ニューラルネットワークのパラメータには単語同士の関係性が学習されることが期待されます。

日本で最も高い山は「？」の？を当てるために、
- ？には文法的に名詞が入る
- その候補は文脈的に山であり
- 日本で最も高いという情報がある、というような文脈上のどの情報に着目するか
- 文脈を踏まえた上で今まで得た知識をどのように組み合わせるか

ということをニューラルネットワークが学習します。

![NNLM](images/nnlm.png)

### MLPによるモデル化

![MLPLM](images/mlplm.png)  
CS224N: Natural Language Processing with Deep Learningより引用

In [ ]:
# MLPの学習のためのデータの準備
import torch
import torch.nn as nn
from torch.nn import functional as F
from matplotlib import pyplot as plt
%matplotlib inline

seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

sentences = clean_text('neco.html')
chars = sorted(list(set(''.join(sentences))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['<E>'] = 0
itos = {i:s for s,i in stoi.items()}

# build the dataset
window_size = 4

def build_dataset(sentences, window_size):  
  X, Y, total_words = [], [], []
  for w in sentences:
    context = [0] * window_size
    for ch in w:
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      total_words.append(w)
      context = context[1:] + [ix] # crop and append
    ix = stoi['<E>']
    X.append(context)
    Y.append(ix)
    total_words.append(w + '<E>')

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  return X, Y, total_words

import random
random.seed(42)
random.shuffle(sentences)
n1 = int(0.99*len(sentences))
n2 = int(0.995*len(sentences))

Xtr, Ytr, sentencetr = build_dataset(sentences[:n1], window_size)
Xdev, Ydev, _ = build_dataset(sentences[n1:n2], window_size)
Xte, Yte, _ = build_dataset(sentences[n2:], window_size)

sample_idx = 4
example_num = 3
embedding_dim = 3
embedding_table = torch.randn((len(stoi), embedding_dim))

print(sentencetr[sample_idx])
for n in range(example_num):
    print([itos[idx] for idx in Xtr[sample_idx + n].tolist()], '---MLP--->', itos[Ytr[sample_idx + n].item()])
    print(Xtr[sample_idx + n].tolist(), '---MLP--->', Ytr[sample_idx + n].item())
    for idx, embedding in enumerate(embedding_table[Xtr[sample_idx + n].tolist()].tolist()):
        if idx == int(window_size / 2):
            print(embedding, '  ----------MLP---------->', Ytr[sample_idx + n].item())
        else:
            print(embedding)
    print('--'*20)

In [ ]:
# MLPの定義
class MLP(nn.Module):
    def __init__(self, window_size, vocab_size, n_embd, hidden_dim):
        super().__init__()
        self.window_size = window_size
        self.vocab_size = vocab_size
        self.wte = nn.Embedding(vocab_size, n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(self.window_size * n_embd, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, self.vocab_size)
        )

    def forward(self, idx):
        embs = self.wte(idx) # (b, window_size, n_embd)
        batch_size, window_size, n_embd = embs.shape
        x = embs.view(batch_size, -1) # (b, window_size, n_embd * window_size)
        logits = self.mlp(x)
        return logits

#### 演習1: MLPの層を増やしたり、学習率などを変更して未来の単語の予測精度向上を試みる

In [ ]:
batch_size = 512
total_step_num = 10
window_size = 4
embedding_dim = 64
hidden_dim = 256

mlp = MLP(window_size, len(stoi), embedding_dim, hidden_dim)
mlp.train()
optimizer = torch.optim.AdamW(mlp.parameters(), lr=5e-4, weight_decay=0.01, betas=(0.9, 0.99), eps=1e-8)
lossi = []
stepi = []
for step in range(total_step_num):
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    logits = mlp(Xtr[ix])
    loss = F.cross_entropy(logits, Ytr[ix])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    stepi.append(step)
    lossi.append(loss.log10().item())
# 最終的なloss
print(loss.log10().item())
# lossの推移
plt.plot(stepi, lossi)

In [ ]:
# MLP言語モデルによる生成
g = torch.Generator().manual_seed(1)

out = []
context_str = '吾輩（わ'
context_idx = [stoi[ch] for ch in context_str]
print('入力: ', context_str)
while True:
    x = torch.tensor(context_idx).unsqueeze(0)
    logits = mlp(x)
    ix = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1, replacement=True, generator=g).item()
    context_idx = context_idx[1:] + [ix]
    if ix == 0 or len(out) >= 100:
        break
    out.append(ix)
print('生成結果: ', context_str + ''.join(itos[i] for i in out))

言語ベクトルとMLPによるモデル化によって
- 密な言語ベクトル表現によって、単語の意味や概念を表現
- パラメータ数がO(exp(n))からO(n)へと削減

残る課題
- 見れる過去の文脈長が固定、増やそうとするとパラメータ数が増加

### RNNによるモデル化

In [ ]:
import torch

vocab = {'あ': 0, 'い': 1, 'う':2, 'え':3, 'お':4, '<sos>':5, '<eos>':6}
idx_to_ch = dict((v, k) for (k,v) in vocab.items())
text = 'いえい'
text = ['<sos>'] + list(text) + ['<eos>']
text = [vocab[ch] for ch in text]
model_input = torch.tensor(text[:-1])
model_target = torch.tensor(text[1:])
print('model input', model_input.tolist(), [idx_to_ch[idx] for idx in model_input.tolist()])
print('model target', model_target.tolist(), [idx_to_ch[idx] for idx in model_target.tolist()])

In [ ]:
# RNNの定義
vocab_size = len(vocab)
embedding_dim = 5
hidden_dim = 3
hidden_start = torch.zeros((1, hidden_dim)).T
word_embedding_table = torch.randn((vocab_size, embedding_dim))

We = torch.randn((hidden_dim, embedding_dim))
Wh = torch.randn((hidden_dim, hidden_dim))
Wy = torch.randn((vocab_size, hidden_dim))

In [ ]:
# RNNの順伝播の計算の様子
h_t_minus_1 = hidden_start
input_history = []

# 前のステップの計算が次のステップの計算に影響するため並列化が難しい。
# RNNの計算複雑度 len(model_input.tolist()) * hidden_dim * hidden_dim = T * d * d
for t in range(len(model_input.tolist())):
    idx = model_input.tolist()[t]
    input_history.append(idx_to_ch[idx])
    print(input_history, '----> RNN ----> ', idx_to_ch[model_target.tolist()[t]])
    x = torch.LongTensor([idx])
    x = word_embedding_table[x].T
    # Attentionを導入したいポイント、過去の文脈がh_t_minus_1に押し込まれる
    # ネットワークが単語方向に深くなるため学習が不安定に
    # d * d, 系列長によらず一定
    h_t = torch.tanh(torch.matmul(We, x) + torch.matmul(Wh, h_t_minus_1))
    logits = torch.matmul(Wy, h_t)
    print(f'P({idx_to_ch[model_target.tolist()[t]]} | {", ".join(input_history)}) = {torch.softmax(logits, dim=0).squeeze().tolist()[model_target.tolist()[t]]:.3f}')
    model_target_onehot = torch.zeros((1, vocab_size))
    model_target_onehot[0, model_target.tolist()[t]] = 1
    h_t_minus_1 = h_t
    output_dist = [float('{:.3f}'.format(output)) for output in torch.softmax(logits, dim=0).squeeze().tolist()]
    print(f'{output_dist} <-----学習によって近づける-----> {model_target_onehot.tolist()[0]}')
    print()

RNNによるモデル化によって
- 文脈長を固定せずに、任意の長さの文脈を考慮
- 文脈長が増えてもパラメータ数が増えない

残る課題
- 学習が不安定(勾配消失、勾配爆発)
- 並列化ができず、学習が遅い
- 文脈長が長くなるとトークンの長距離依存関係の把握が難しくなる。

### Transformerによるモデル化と学習

![Transformer](images/transformer_intro.png)

![RNN computation](images/rnn_computation.png)  
CS224N: Natural Language Processing with Deep Learningより引用

![Attention computation](images/attention_computation.png)  
CS224N: Natural Language Processing with Deep Learningより引用

![Transformer](images/transformer.png)  
Attention Is All You Needより引用

Attention $(Q, K, V)=\operatorname{softmax}\left(\frac{Q K^T}{\sqrt{d_k}}\right) V$

Attention Is All You Needの式(1)より

$\operatorname{FFN}(x)=\max \left(0, x W_1+b_1\right) W_2+b_2$  
Attention Is All You Needの式(2)より

In [ ]:
import torch
import time
vocab = {'あ': 0, 'い': 1, 'う':2, 'え':3, 'お':4, '<sos>':5, '<eos>':6}
idx_to_ch = dict((v, k) for (k,v) in vocab.items())
text = 'いえい'
text = ['<sos>'] + list(text) + ['<eos>']
text = [vocab[ch] for ch in text]
model_input = torch.tensor(text[:-1])
model_target = torch.tensor(text[1:])
print('model input', model_input.tolist(), [idx_to_ch[idx] for idx in model_input.tolist()])
print('model target', model_target.tolist(), [idx_to_ch[idx] for idx in model_target.tolist()])


vocab_size = len(vocab)
d_model = embedding_dim = 3 # Attention is all you needの論文では512
d_ff = d_model * 4
n_head = 1 # Attention is all you needの論文では8
d_k = int(d_model / n_head)
d_v = int(d_model / n_head)

word_embedding_table = torch.randn((vocab_size, embedding_dim))

Wq = torch.randn((d_k, embedding_dim))
Wk = torch.randn((d_k, embedding_dim))

Wv = torch.randn((d_v, embedding_dim))
Wo = torch.randn((d_model, d_v * n_head))

Wff_1 = torch.randn((d_ff, d_model))
Wff_2 = torch.randn((d_model, d_ff))
Wy = torch.randn((vocab_size, d_model))

print('self-attentionのパラメータ数', d_model * d_k * n_head * 2 + d_model * d_v * n_head + d_model * d_v * n_head)
print('self-attentionのWvのパラメータ数', d_model * d_v * n_head)
print('feed-forwardのパラメータ数', d_ff * d_model * 2)

In [ ]:
# 並列化されていないSelf-Attention層、FeedForward層の計算
quries = []
keys = []
values = []
attention_scores = []
attention_outputs = []
input_history = []

# Self-Attentionの計算複雑度 len(model_input.tolist()) * len(model_input.tolist()) * d_model = T * T * d
for t in range(len(model_input.tolist())):
    idx = model_input.tolist()[t]
    input_history.append(idx_to_ch[idx])
    x = torch.LongTensor([idx])
    x = word_embedding_table[x].T # + positional_encodings
    
    # single-head, multi-headになると複数の観点でquery, key, valueの発行をする
    query_t = torch.matmul(Wq, x) # 〇〇探してます！ 例: token_0: 自分主語です、目的語とか助詞とか動詞とか探してます！

    key_t = torch.matmul(Wk, x) # 〇〇持ってます！ 例: token_0: 自分主語です！ token_1: 自分動詞です!
    value_t = torch.matmul(Wv, x) # 中身の詳細です！ 例: token_0: 「拙者」です、珍しい1人称です、お侍さんとかが使ったりします token_1: 「食べる」です、食べ物を口に入れる行為です
    
    # cross attentionの場合はkey, valueは
    # key_t = torch.matmul(Wk, another_modality_x)
    # value_t = torch.matmul(Wv, another_modality_x)のようになる

    quries.append(query_t)
    keys.append(key_t)
    values.append(value_t)
    # Day2の演習では文章単位でベクトル化したqueryとkeyの内積(類似度)をもとにRetrievalを行った
    # 計算複雑度 T * d、系列長によって変化
    # 遠く離れた過去の文脈を考慮できる、入力データに応じて相互作用し、その結果が後の深い層でも反映される
    attention_score = torch.matmul(query_t.T, torch.stack(keys)) / torch.sqrt(torch.tensor(d_k))
    attention_score = torch.softmax(attention_score, dim=0)
    attention_scores.append(attention_score.squeeze())
    self_attention_output = 0
    for i in range(len(attention_score)):
        # query、keyのコミュニケーションの結果がvalueに反映される(attention scoreで重み付け)
        self_attention_output += attention_score[i] * values[i]
    attention_outputs.append(self_attention_output)

    # FeedForwardの計算
    ff_output = torch.matmul(Wff_2, torch.relu(torch.matmul(Wff_1, self_attention_output)))
    # 次単語予測、タスク特化のための分類器を作るような場合には新しくWyを用意して学習する
    logits = torch.matmul(Wy, ff_output)
    output_dist = [float('{:.3f}'.format(output)) for output in torch.softmax(logits, dim=0).squeeze().tolist()]
    
    print(f'P({idx_to_ch[model_target.tolist()[t]]} | {", ".join(input_history)}) = {torch.softmax(logits, dim=0).squeeze().tolist()[model_target.tolist()[t]]:.3f}')
    model_target_onehot = torch.zeros((1, vocab_size))
    model_target_onehot[0, model_target.tolist()[t]] = 1
    print(f'{output_dist} <-----学習によって近づける-----> {model_target_onehot.tolist()[0]}')
    print()

In [ ]:
# attentionの可視化
attention_map = torch.zeros((len(model_input.tolist()), len(model_input.tolist())))
for t in range(len(model_input.tolist())):
    attention_map[t][:t+1] = attention_scores[t]
    print('token', input_history)
    print(input_history[t], attention_map[t].tolist())
    print('-----' * 20)


In [ ]:
# self-attentionで必要になるメモリ len(model_input.tolist()) * len(model_input.tolist()) = T * T
print(attention_map.shape)

In [ ]:
# 未来の情報を用いないようにCausal Attentionを用いて並列化(行列の計算にする)
input_ids = model_input.tolist()
x = torch.LongTensor([input_ids])
x = word_embedding_table[x] # + positional_encodings
# print(x.shape) # (1, T, embedding_dim)

queries = torch.matmul(x.squeeze(), Wq.T) # (T, d_k)
keys = torch.matmul(x.squeeze(), Wk.T) # (T, d_k)
values = torch.matmul(x.squeeze(), Wv.T) # (T, d_v)
# print(queries.shape, keys.shape, values.shape)

attention_scores = torch.matmul(queries, keys.T) / torch.sqrt(torch.tensor(d_k)) # (T, T)
# causal attention、未来の情報を用いないようにする
attention_mask = torch.tril(torch.ones((len(model_input.tolist()), len(model_input.tolist())))) # (T, T)
attention_scores = attention_scores.masked_fill(attention_mask==0, float('-inf'))

attention_scores = torch.softmax(attention_scores, dim=1) # (T, T)
attention_outputs = torch.matmul(attention_scores, values) # (T, d_v)

# FeedForwardの計算
ff_output = torch.matmul(torch.relu(torch.matmul(attention_outputs, Wff_1.T)), Wff_2.T) # (T, d_model)
# print(ff_output.shape)
# 次単語予測
logits = torch.matmul(ff_output, Wy.T) # (T, vocab_size)
output_dists = torch.softmax(logits, dim=1) # (T, vocab_size)

for step_t, output_dist in enumerate(output_dists):
    print(f'P({idx_to_ch[model_target.tolist()[step_t]]} | {", ".join(input_history[:step_t + 1])}) = {output_dist[model_target.tolist()[step_t]]:.3f}')
    model_target_onehot = torch.zeros((1, vocab_size))
    model_target_onehot[0, model_target.tolist()[step_t]] = 1
    print(f'{[float("{:.3f}".format(output)) for output in output_dist.tolist()]} <-----学習によって近づける-----> {model_target_onehot.tolist()[0]}')
    print()


In [ ]:
attention_scores = torch.matmul(queries, keys.T) / torch.sqrt(torch.tensor(d_k)) # (T, T)
print('causal attention前')
print(attention_scores)
# causal attentionの場合はattention_maskを用いて未来の情報をマスクする
attention_mask = torch.tril(torch.ones((len(model_input.tolist()), len(model_input.tolist())))) # (T, T)
# 下三角行列で未来の情報をマスク
print(attention_mask)
# 未来の情報はscoreが0になるように-infを代入(softmaxで0になる)
attention_scores = attention_scores.masked_fill(attention_mask==0, float('-inf'))
print('causal attention後')
print(attention_scores)
attention_scores = torch.softmax(attention_scores, dim=1) # (T, T)
print('softmax後')
print(attention_scores)

上記の実装に加え、学習の安定化のためにResidual ConnectionとLayer Normalizationを追加することでTransformerのblockを実装できます。
以下にGPT-2の実装と学習のコードを示します。

In [ ]:
# GPT-2の実装 code from https://github.com/karpathy/makemore/tree/master
# https://github.com/karpathy/makemore/blob/master/makemore.py
"""
MIT License

Copyright (c) 2022 Andrej Karpathy

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""
import os
import sys
import time
import math
import argparse
from dataclasses import dataclass
from typing import List

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

# -----------------------------------------------------------------------------

@dataclass
class ModelConfig:
    block_size: int = None # length of the input sequences of integers
    vocab_size: int = None # the input integers are in range [0 .. vocab_size -1]
    # parameters below control the sizes of each model slightly differently
    n_layer: int = 4
    n_embd: int = 64
    n_embd2: int = 64
    n_head: int = 4

# -----------------------------------------------------------------------------
# Transformer Language Model (*exactly* as used in GPT-2)

class NewGELU(nn.Module):
    """
    Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT).
    Reference: Gaussian Error Linear Units (GELU) paper: https://arxiv.org/abs/1606.08415
    """
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

class CausalSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    It is possible to use torch.nn.MultiheadAttention here but I am including an
    explicit implementation here to show that there is nothing too scary here.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k ,v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.c_proj(y)
        return y

class Block(nn.Module):
    """ an unassuming Transformer block """

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.ModuleDict(dict(
            c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd),
            c_proj  = nn.Linear(4 * config.n_embd, config.n_embd),
            act     = NewGELU(),
        ))
        m = self.mlp
        self.mlpf = lambda x: m.c_proj(m.act(m.c_fc(x))) # MLP forward

    def forward(self, x):
        # residual connection
        x = x + self.attn(self.ln_1(x))
        # residual connection
        x = x + self.mlpf(self.ln_2(x))
        return x

class Transformer(nn.Module):
    """ Transformer Language Model, exactly as seen in GPT-2 """

    def __init__(self, config):
        super().__init__()
        self.block_size = config.block_size

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # report number of parameters (note we don't count the decoder parameters in lm_head)
        n_params = sum(p.numel() for p in self.transformer.parameters())
        print("トランスフォーマーのパラメーター数: %.2fM" % (n_params/1e6,))

    def get_block_size(self):
        return self.block_size

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.block_size, f"Cannot forward sequence of length {t}, block size is only {self.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0) # shape (1, t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (1, t, n_embd)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)

        # if we are given some desired targets also calculate the loss
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)

        return logits, loss

#### 演習2: Transformerのモデルサイズなどのパラメータを変更して未来の単語の予測精度向上を試みる

In [ ]:
from utils import create_datasets, InfiniteDataLoader
train_dataset, test_dataset = create_datasets('neco.txt')
vocab_size = train_dataset.get_vocab_size()
block_size = train_dataset.get_output_length()

############ ハイパーパラメーター
n_layer = 4
n_head = 4
n_embd = 64
n_embd2 = 64
lr = 5e-4
weight_decay = 0.01
batch_size = 2
max_steps = 10
# ハイパーパラメーター ############

config = ModelConfig(
    vocab_size=vocab_size, 
    block_size=block_size,
    n_layer=n_layer,
    n_head=n_head,
    n_embd=n_embd, 
    n_embd2=n_embd2
)

model = Transformer(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# init optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay, betas=(0.9, 0.99), eps=1e-8)

# init dataloader
batch_loader = InfiniteDataLoader(train_dataset, batch_size=batch_size, pin_memory=True, num_workers=4)

# training loop
best_loss = None
step = 0
stepi = []
lossi = []
while True:
    # get the next batch, ship to device, and unpack it to input and target
    batch = batch_loader.next()
    batch = [t.to(device) for t in batch]
    X, Y = batch

    # feed into the model
    logits, loss = model(X, Y)

    # calculate the gradient, update the weights
    model.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    t1 = time.time()

    stepi.append(step)
    lossi.append(loss.log10().item())
    step += 1
    # termination conditions
    if max_steps >= 0 and step >= max_steps:
        break
plt.plot(stepi, lossi)

In [ ]:
# Transoformer言語モデルによる生成
from utils import generate
context = '吾輩（わ'
print('入力: ', context)
X_init = train_dataset.encode(context).to(device).unsqueeze(0)
top_k = 1
# steps = train_dataset.get_output_length() - 1 
steps = 20
X_samp = generate(model, X_init, steps, top_k=top_k, do_sample=True).to('cpu')
row = X_samp[0].tolist()
crop_index = row.index(0) if 0 in row else len(row)
row = row[:crop_index]
print('生成結果: ', train_dataset.decode(row)[:100])


## 参考
- [Andrej Karpathy(元TeslaのAIチームのリーダー、現在はOpenAI)によるGPT-2実装までの講義動画](https://www.youtube.com/watch?v=VMj-3S1tku0&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ)
- [CS224N: Natural Language Processing with Deep Learning](http://web.stanford.edu/class/cs224n/)
    - https://web.stanford.edu/class/cs224n/slides/cs224n-2023-lecture08-transformers.pdf